# Data Preprocessing and Feature Engineering

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [8]:
data = pd.read_csv('data_traffic_congestion_2020.csv')

In [9]:
data.head()

,Date,Day,Destination_Location,Fastest_Route_Distance,Fastest_Route_Name,Fastest_Route_Time,Holiday,Special_Condition,Starting_Location,Sys_Time,Weather,Data_prediction
0,13/02/2020,Thursday,9th_avenue,1000,Jinnah Avenue,60,no,no,capital_gymkhana,0:00:01,Showers,smooth
1,13/02/2020,Thursday,centaurus,1300,Jinnah Avenue,120,no,no,9th_avenue,0:00:09,Showers,highly congested
2,13/02/2020,Thursday,crown_plaza,1000,Jinnah Avenue and Faisal Avenue Flyover,60,no,no,centaurus,0:00:17,Showers,smooth
3,13/02/2020,Thursday,roshan_plaza,2700,Jinnah Ave,180,no,no,crown_plaza,0:00:24,Showers,smooth
4,13/02/2020,Thursday,salt_and_pepper,3900,Jinnah Ave,300,no,no,roshan_plaza,0:00:31,Showers,slightly congested


#### Creating features for Date

In [10]:
data['Date'] = pd.to_datetime(data.Date, format='%d/%m/%Y')

In [11]:
def create_date_features(df):
    df['month'] = df.Date.dt.month
    df['day_of_month'] = df.Date.dt.day
    df['day_of_year'] = df.Date.dt.dayofyear
    df['week_of_year'] = df.Date.dt.weekofyear
    df['year'] = df.Date.dt.year
    #check
    df["is_wknd"] = df.Date.dt.weekday // 4
    df['is_month_start'] = df.Date.dt.is_month_start.astype(int)
    df['is_month_end'] = df.Date.dt.is_month_end.astype(int)
    return df

In [12]:
#convertig to category reduces data space required
data["Destination_Location"]=data["Destination_Location"].astype("category")
data["Day"]=data["Day"].astype("category")
data["Fastest_Route_Name"]=data["Fastest_Route_Name"].astype("category")
data["Starting_Location"]=data["Starting_Location"].astype("category")
data["Weather"]=data["Weather"].astype("category")

In [13]:
data = create_date_features(data)

<ipython-input-11-4a91150b0670>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df.Date.dt.weekofyear


In [14]:
data

,Date,Day,Destination_Location,Fastest_Route_Distance,Fastest_Route_Name,Fastest_Route_Time,Holiday,Special_Condition,Starting_Location,Sys_Time,Weather,Data_prediction,month,day_of_month,day_of_year,week_of_year,year,is_wknd,is_month_start,is_month_end
0,2020-02-13,Thursday,9th_avenue,1000,Jinnah Avenue,60,no,no,capital_gymkhana,0:00:01,Showers,smooth,2,13,44,7,2020,0,0,0
1,2020-02-13,Thursday,centaurus,1300,Jinnah Avenue,120,no,no,9th_avenue,0:00:09,Showers,highly congested,2,13,44,7,2020,0,0,0
2,2020-02-13,Thursday,crown_plaza,1000,Jinnah Avenue and Faisal Avenue Flyover,60,no,no,centaurus,0:00:17,Showers,smooth,2,13,44,7,2020,0,0,0
3,2020-02-13,Thursday,roshan_plaza,2700,Jinnah Ave,180,no,no,crown_plaza,0:00:24,Showers,smooth,2,13,44,7,2020,0,0,0
4,2020-02-13,Thursday,salt_and_pepper,3900,Jinnah Ave,300,no,no,roshan_plaza,0:00:31,Showers,slightly congested,2,13,44,7,2020,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317107,2020-02-29,Saturday,IJP road,4400,Murree Rd and I.J.P. Road,540,no,no,mehran hotel,23:57:23,Mostly Cloudy,slightly congested,2,29,60,9,2020,1,0,1
317108,2020-02-29,Saturday,Jamia Masjid Bilal,1000,I.J.P. Road,180,no,no,IJP road,23:57:36,Mostly Cloudy,congested,2,29,60,9,2020,1,0,1
317109,2020-02-29,Saturday,eventox_event_management,1000,Sufi Tabasum Rd and Service Rd W,120,no,no,STroad_start_stop,23:57:46,Mostly Cloudy,highly congested,2,29,60,9,2020,1,0,1
317110,2020-02-29,Saturday,Al_hateem_palace,1100,Service Rd South I 8,120,no,no,IJP_metro,23:58:18,Mostly Cloudy,highly congested,2,29,60,9,2020,1,0,1


### Removing outliers

In [24]:
# functions to treat outliers by flooring and capping


def treat_outliers(df, col):
    """
    Treats outliers in a variable

    df: dataframe
    col: dataframe column
    """
    Q1 = df[col].quantile(0.25)  # 25th quantile
    Q3 = df[col].quantile(0.75)  # 75th quantile
    IQR = Q3 - Q1
    Lower_Whisker = Q1 - 1.5 * IQR
    Upper_Whisker = Q3 + 1.5 * IQR

    # all the values smaller than Lower_Whisker will be assigned the value of Lower_Whisker
    # all the values greater than Upper_Whisker will be assigned the value of Upper_Whisker
    df[col] = np.clip(df[col], Lower_Whisker, Upper_Whisker)

    return df


def treat_outliers_all(df, col_list):
    """
    Treat outliers in a list of variables

    df: dataframe
    col_list: list of dataframe columns
    """
    for c in col_list:
        df = treat_outliers(df, c)

    return df

In [25]:
numerical_col = data.select_dtypes(include=np.number).columns.tolist()
data = treat_outliers_all(data, numerical_col)

#### Reducing Number of Disntinct Unique values in Data Prediction

In [15]:
congested = [ "highly congested", "slightly congested", "congested", "blockage"]
smooth = ["smooth"]

def reduce_marital_status(x):
    if x in congested:
        return "congested"
    elif x in smooth:
        return "smooth"
    else:
        return x

In [16]:
data.Data_prediction = data.Data_prediction.apply(reduce_marital_status)

In [17]:
data.Data_prediction.unique()

array(['smooth', 'congested'], dtype=object)

### One Hot Encoding

In [18]:
data = pd.get_dummies(data, columns=['Destination_Location','Day','Fastest_Route_Name','Starting_Location','Weather'])
data.head()

,Date,Fastest_Route_Distance,Fastest_Route_Time,Holiday,Special_Condition,Sys_Time,Data_prediction,month,day_of_month,day_of_year,...,Starting_Location_zaman_khan_khokha,Weather_Clear,Weather_Cloudy,Weather_Mostly Clear,Weather_Mostly Cloudy,Weather_Mostly Sunny,Weather_Partly Cloudy,Weather_Rain,Weather_Showers,Weather_Sunny
0,2020-02-13,1000,60,no,no,0:00:01,smooth,2,13,44,...,0,0,0,0,0,0,0,0,1,0
1,2020-02-13,1300,120,no,no,0:00:09,congested,2,13,44,...,0,0,0,0,0,0,0,0,1,0
2,2020-02-13,1000,60,no,no,0:00:17,smooth,2,13,44,...,0,0,0,0,0,0,0,0,1,0
3,2020-02-13,2700,180,no,no,0:00:24,smooth,2,13,44,...,0,0,0,0,0,0,0,0,1,0
4,2020-02-13,3900,300,no,no,0:00:31,congested,2,13,44,...,0,0,0,0,0,0,0,0,1,0


In [19]:
data.head()

,Date,Fastest_Route_Distance,Fastest_Route_Time,Holiday,Special_Condition,Sys_Time,Data_prediction,month,day_of_month,day_of_year,...,Starting_Location_zaman_khan_khokha,Weather_Clear,Weather_Cloudy,Weather_Mostly Clear,Weather_Mostly Cloudy,Weather_Mostly Sunny,Weather_Partly Cloudy,Weather_Rain,Weather_Showers,Weather_Sunny
0,2020-02-13,1000,60,no,no,0:00:01,smooth,2,13,44,...,0,0,0,0,0,0,0,0,1,0
1,2020-02-13,1300,120,no,no,0:00:09,congested,2,13,44,...,0,0,0,0,0,0,0,0,1,0
2,2020-02-13,1000,60,no,no,0:00:17,smooth,2,13,44,...,0,0,0,0,0,0,0,0,1,0
3,2020-02-13,2700,180,no,no,0:00:24,smooth,2,13,44,...,0,0,0,0,0,0,0,0,1,0
4,2020-02-13,3900,300,no,no,0:00:31,congested,2,13,44,...,0,0,0,0,0,0,0,0,1,0


In [20]:
data.drop(["Holiday","Special_Condition","Sys_Time","Date"], axis = 1, inplace = True)

### Export Data for Model Training

In [21]:
X = data.drop(["Data_prediction"], axis=1)
y = data["Data_prediction"]

In [22]:
X.columns

Index(['Fastest_Route_Distance', 'Fastest_Route_Time', 'month', 'day_of_month',
       'day_of_year', 'week_of_year', 'year', 'is_wknd', 'is_month_start',
       'is_month_end',
       ...
       'Starting_Location_zaman_khan_khokha', 'Weather_Clear',
       'Weather_Cloudy', 'Weather_Mostly Clear', 'Weather_Mostly Cloudy',
       'Weather_Mostly Sunny', 'Weather_Partly Cloudy', 'Weather_Rain',
       'Weather_Showers', 'Weather_Sunny'],
      dtype='object', length=143)

In [23]:
X.to_csv('X.csv', index=False)
y.to_csv('y.csv', index=False)